# TextRank



- 각 행렬에서, 행의 합은 원래 score
- 열의 합이 새로운 Score
- 행으로 보고, 연결된 edge의 수를 score로나누면 행을 채울수 있고
- 만들어진 열을 더하면 새로운 스코어가나온다

- 만들어진 score랑 새 score의차이가 0.0001인가 아니면 조금 더 적으면(20~30번 반복해서)  stop. 한글은 다르겠지?

- 만약 window가 3이라면, 123 -> 12 :1, 13:1, 23 :1, 중복 X 무적권 연결이 중요한거니 1로 한다.


In [2]:
!pip install mecab_python-0.996_ko_0.9.2_msvc-cp36-cp36m-win_amd64.whl

In [2]:
import MeCab

In [3]:
from konlpy.tag import Mecab

In [3]:
import MeCab
from konlpy.tag import Mecab
mecab = Mecab(dicpath = 'C:\mecab\mecab-ko-dic')

### 2) Unique한 토큰 목록 생성

그래프 생성을 위해서 Unique한 토큰 목록 생성

In [3]:
token = ['딸기', '바나나', '사과', '딸기','파인애플']
nodes = ['바나나', '사과', '파인애플', '딸기']
vocab = nodes

In [4]:
from collections import defaultdict

vocab2idx = {vocab[i] : i for i in range(len(vocab))}
idx2vocab = {i : vocab[i] for i in range(len(vocab))}

### 3) 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex) 
*   weighted_edge 행렬은 노드간 가중치 정보를 담고 있음
*   weighted_edge[i][j] 는 i번째 단어와 j번째 단어의 가중치를 의미
*   weighted_edge[i][j] 가 0인 경우는 노드간 연결이 없음을 의미
*   모든 노드는 1로 초기화

In [8]:
import numpy as np
import math
vocab_len = len(vocab)

zero_list = np.zeros((vocab_len, vocab_len), dtype = np.float32)
# 토큰별로 그래프 edge를 Matrix 형태로 생성
for i in range(len(token)):
    for j in range(len(token)):
        if np.abs(i-j) == 1:
            zero_list[vocab2idx[token[i]]][vocab2idx[token[j]]] = 1
      
    
# weighted_edge = np.ones((vocab_len, vocab_len))

# 각 토큰 노드별로 스코어 1로 초기화
# score = None

# coocurrence를 판단하기 위한 window 사이즈 설정
# window_size = 2
# covered_coocurrences = []

# weighted_edge 구현



In [20]:
zero_list

array([[0.        , 0.5       , 0.        , 0.5       ],
       [0.5       , 0.        , 0.        , 0.5       ],
       [0.        , 0.        , 0.        , 1.        ],
       [0.33333334, 0.33333334, 0.33333334, 0.        ]], dtype=float32)

In [21]:
for i in range(len(zero_list)):
    zero_list[i] = zero_list[i] /zero_list[i].sum()

In [22]:
zero_list

array([[0.        , 0.5       , 0.        , 0.5       ],
       [0.5       , 0.        , 0.        , 0.5       ],
       [0.        , 0.        , 0.        , 1.        ],
       [0.33333334, 0.33333334, 0.33333334, 0.        ]], dtype=float32)

In [23]:
score

array([1.01384075, 1.01384075, 0.5233411 , 1.44897774])

### 4) 각 노드의 score계산
각 노드와 연결된 weighted edge의 값을 합산

In [12]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

score = np.array([1,1,1,1])
for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    for i in range(len(score)):
        score = zero_list.T @ score
    if score[0] - prev_score[0] <= threshold:
         break


In [13]:
score, prev_score

(array([1.01384075, 1.01384075, 0.5233411 , 1.44897774]),
 array([1.04861116, 1.04861116, 0.58333337, 1.31944449]))

### 5) 핵심 단어 추출

In [14]:
sorted_index = { i :np.argsort(-score)[i]for  i in range(len(score))}
n = 4

print("\n=== 핵심키워드 ===")
for i in range(0,n):
    print(str(idx2vocab[sorted_index[i]])+" : " + str(score[sorted_index[i]]))


=== 핵심키워드 ===
딸기 : 1.4489777411196274
바나나 : 1.0138407546975161
사과 : 1.0138407546975161
파인애플 : 0.5233411024048669


In [ ]:
import numpy as np
import math
vocab_len = len(vocab)

zero_list = np.zeros((vocab_len, vocab_len), dtype = np.float32)
# 토큰별로 그래프 edge를 Matrix 형태로 생성
for i in range(len(token)):
    for j in range(len(token)):
        if np.abs(i-j) == 1:
            zero_list[vocab2idx[token[i]]][vocab2idx[token[j]]] = 1
      
    
# weighted_edge = np.ones((vocab_len, vocab_len))

# 각 토큰 노드별로 스코어 1로 초기화
# score = None

# coocurrence를 판단하기 위한 window 사이즈 설정
# window_size = 2
# covered_coocurrences = []

# weighted_edge 구현


이제 윈도우 사이즈가 주어졌을때를 생각해보자..

만약 원래 테스트 길이가 5고, window size=2 라면

12, 23, 34, 45 총4번

만약 window size = 3이라면

123, 234, 345 이렇게 3번

그럼 3번 뽑고,  window size만큼 슬라이싱

그럼 [1,2,3], [2,3,4], [3,4,5] 가 뽑혔으면
12 13 23 에 +1 에 역으로인덱스를 key로 받는거 해서 token[]을 정하고 거기에 += 1
23 24 34에 +=1 


In [19]:
window_size = 2
token_len = len(token)
new_token = []

for i in range(token_len - window_size + 1):
    new_token.append(token[i:i+window_size])


[['딸기', '바나나'], ['바나나', '사과'], ['사과', '딸기'], ['딸기', '파인애플']]

In [ ]:
for tok in new_token:
    for i in range(len(window_size - 1)):
        

# 선생님이 한 방법

In [24]:
token = ['딸기', '바나나', '사과', '딸기','파인애플']
nodes = ['바나나', '사과', '파인애플', '딸기']
vocab = nodes

dict_comprehension  

vocab2idx = {}  
for i in range(len(vocab)):  
    vocab2idx[vocab[i]] = i  

-->  vocab2idx = { vocab[i] : i for i in range(len(vocab))}
    

In [26]:
vocab2idx = {vocab[i] : i for i in range(len(vocab))}
idx2vocab = {i : vocab[i] for i in range(len(vocab))}
vocab2idx

{'바나나': 0, '사과': 1, '파인애플': 2, '딸기': 3}

### 3) 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex) 
*   weighted_edge 행렬은 노드간 가중치 정보를 담고 있음
*   weighted_edge[i][j] 는 i번째 단어와 j번째 단어의 가중치를 의미
*   weighted_edge[i][j] 가 0인 경우는 노드간 연결이 없음을 의미
*   모든 노드는 1로 초기화

In [27]:
range(4,4)

range(4, 4)

In [13]:
import numpy as np
import math
vocab_len = len(vocab)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.zeros((vocab_len, vocab_len))

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((vocab_len))

# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 2
covered_coocurrences = []

# weighted_edge 구현
for window_start in range(len(token) - window_size + 1):
    window = token[window_start : window_start + window_size]
    for i in range(window_size):
        for j in range(i+1, window_size):
            if (window[i] in vocab2idx.keys()) and (window[j] in vocab2idx.keys()):     #만약 동사, 명사만 할건데 전체가 없으면 그만!
                index_i = window_start + i
                index_j = window_start + j
                
                if [index_i, index_j] not in covered_coocurrences:
                    weighted_edge[vocab2idx[window[i]]][vocab2idx[window[j]]] = 1
                    weighted_edge[vocab2idx[window[j]]][vocab2idx[window[i]]] = 1
                    covered_coocurrences.append([index_i, index_j])
    
    
covered_coocurrences

[[0, 1], [1, 2], [2, 3], [3, 4]]

In [74]:
for i in range(vocab_len):
    sumi = weighted_edge[i].sum()
    weighted_edge[i] = weighted_edge[i]/sumi if sumi > 0 else 0

In [75]:
weighted_edge

array([[0.        , 0.5       , 0.        , 0.5       ],
       [0.5       , 0.        , 0.        , 0.5       ],
       [0.        , 0.        , 0.        , 1.        ],
       [0.33333333, 0.33333333, 0.33333333, 0.        ]])

### 4) 각 노드의 score계산
각 노드와 연결된 weighted edge의 값을 합산

In [72]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold


for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(len(vocab)):
        summation = 0
        for j in range(len(vocab)):
            if weighted_edge[j][i] != 0:
                summation += weighted_edge[j][i] * score[j]
            score[i] = (1 -d) + d * (summation)
            
        if np.sum(np.fabs(prev_score - score)) <= threshold:
            break
            


score


array([0.98348008, 0.98342456, 0.56548297, 1.46655755])

### 5)

In [77]:
sorted_index =None
print(np.argsort(score)) # 내림차순을 위해뒤집으려면 [::-1]

[2 1 0 3]


In [78]:
sorted_index =np.argsort(score)[::-1]# 내림차순을 위해뒤집으려면 [::-1]
n=2

for i in range(n):
    print(str(idx2vocab[sorted_index[i]]) + " : " + str(score[sorted_index[i]]))
    

딸기 : 1.46655755344874
바나나 : 0.9834800785071365


In [ ]:
# 이제 이것들을 함수로 만들어야함..

In [6]:
window_size = 3
for window_start in range(len(token) - window_size + 1):
    window = token[window_start : window_start + window_size]
    for i in range(window_size):
        for j in range(i+1, window_size):
            if (window[i] in vocab2idx.keys()) and (window[j] in vocab2idx.keys()):     #만약 동사, 명사만 할건데 전체가 없으면 그만!
                index_i = window_start + 1
                index_j = window_start + 1
                
                if [index_i, index_j] not in covered_coocurrences:
                    weighted_edge[vocab2idx[window[i]]][vocab2idx[window[j]]] = 1
                    weighted_edge[vocab2idx[window[j]]][vocab2idx[window[i]]] = 1
                    covered_coocurrences.append([index_i, index_j])

NameError: name 'covered_coocurrences' is not defined

In [11]:
c = []

c.append([0,0])
c

[[0, 0]]

In [10]:
covered_coocurrences

[[1, 1], [2, 2], [3, 3]]

In [24]:
token = ['딸기', '바나나', '사과', '딸기','파인애플']
nodes = ['바나나', '사과', '파인애플', '딸기']
vocab = nodes

# 그럼 나 혼자 해본다고 생각해봐

# 처음엔 node의 길이(dictionary)만든거 의 shape

vocab2idx = {nodes[i] : i for i in range(len(vocab))}
idx2vocab = {i : nodes[i] for i in range(len(vocab))}

# 일단 weighted_matrix
vocab_len = len(vocab)

weighted_edge = np.zeros((vocab_len, vocab_len))

# score 구현
score = np.ones((vocab_len))

# 겹친거 막기, 그리고 윈도우 사이즈 구현

coocurrence = []
window_size = 2

# 이제 구현해보자

# 우선 window_size에 의해 생기는 경우의 수
# 5개, window 2개 5 - 2 + 1
for window_start in range(len(token) - window_size + 1):
    window = token[window_start : (window_start + window_size)]
    for i in range(window_size):
        for j in range(i+1, window_size):
            if (window[i] in vocab2idx.keys()) and (window[j] in vocab2idx.keys()):
                index_i = window_start + i
                index_j = window_start + j
                
                if [index_i, index_j] not in coocurrence:
                    weighted_edge[vocab2idx[window[i]]][vocab2idx[window[j]]] = 1
                    weighted_edge[vocab2idx[window[j]]][vocab2idx[window[i]]] = 1
                    coocurrence.append([index_i, index_j])

weighted_edge

array([[0., 1., 0., 1.],
       [1., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 1., 1., 0.]])

In [25]:
for i in range(vocab_len):
    weighted_edge[i] = weighted_edge[i]/weighted_edge[i].sum()

weighted_edge

array([[0.        , 0.5       , 0.        , 0.5       ],
       [0.5       , 0.        , 0.        , 0.5       ],
       [0.        , 0.        , 0.        , 1.        ],
       [0.33333333, 0.33333333, 0.33333333, 0.        ]])

In [26]:
# 아니야 했으니까..

MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold


for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(len(vocab)):
        summation = 0
        for j in range(len(vocab)):
            if not weighted_edge[j][i] ==0:
                summation += weighted_edge[j][i] * score[j]
            score[i] = (1-d) + d*(summation)
        if np.sum(np.fabs(score - prev_score)) <= threshold:
            break

            


score


array([0.98348008, 0.98342456, 0.56548297, 1.46655755])

In [34]:
np.argsort(score)

array([2, 1, 0, 3], dtype=int64)

In [40]:
for i in np.argsort(score)[::-1]:
    print('{} : {}'.format(score[i], idx2vocab[i]))

1.46655755344874 : 딸기
0.9834800785071365 : 바나나
0.9834245608548868 : 사과
0.5654829717964912 : 파인애플


In [ ]:
n = 2
for i in range(n):
    print( '{},{}'.format(score[n], idx2vocab(np.argsort(score)[::-1][n])))

In [41]:
np.argsort(score)[::-1][i]

1

In [42]:
idx2vocab[1]

'사과'

In [44]:
n=2
for i in range(n):
    print( '{},{}'.format( idx2vocab(np.argsort(score)[::-1][i]), score[i]))


TypeError: 'dict' object is not callable

In [48]:
idx2vocab(np.argsort(score)[::-1][2])

TypeError: 'dict' object is not callable

In [51]:
np.argsort(score)[::-1][0]

3